In [3]:
import pandas as pd
import pyvista as pv
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt

pv.set_jupyter_backend('trame')  # interactive plotter

file_path = "Data/ManeuverPlan.csv"
df = pd.read_csv(file_path)
df = df.dropna(subset=['WaypointX', 'WaypointY', 'WaypointZ'])
object_positions = np.column_stack((df['WaypointX'], df['WaypointY'], df['WaypointZ']))
max_steps = len(object_positions) - 1
satellite_mesh = pv.read("Models/Dawn.stl")
target_mesh = pv.read("Models/Magellan.stl")


# adjust these numbers to change how the stars look
num_stars = 250
space_range = 5000
star_positions = np.random.uniform(-space_range, space_range, size=(num_stars, 3))
# add stars
star_meshes = []
for pos in star_positions:
    star = pv.Sphere(radius=2, center=pos)
    star_meshes.append(star)
# Combine the star meshes instead of adding one by one
stars_combined = pv.PolyData()
for star in star_meshes:
    stars_combined = stars_combined.merge(star)

# target mesh
target_position = np.array([0, 0, 0])
target = target_mesh.copy()
target.scale(30.0, inplace=True)  # adjust scale here for better visibility
target.translate(target_position, inplace=True)

def plot_maneuver(step, prev_step_range=0, future_step_range=0,
                  show_missed=False, missed_path_length=500, collision_threshold=100,
                  match_width_with_threshold=False):
    plotter = pv.Plotter(notebook=True)
    plotter.set_background("black")

    # Add stars and target
    plotter.add_mesh(stars_combined, color="white", opacity=0.5)
    plotter.add_mesh(target, color="orange", opacity=0.8, name="Target")

    # Satellite position
    current_position = object_positions[step]
    satellite_copy = satellite_mesh.copy()
    satellite_copy.scale(30.0, inplace=True)
    satellite_copy.translate(current_position, inplace=True)
    direction_vector = target_position - current_position
    satellite_copy.rotate_x(np.arctan2(direction_vector[2], direction_vector[1]) * 180 / np.pi)
    plotter.add_mesh(satellite_copy, color="cyan", opacity=1, name="Satellite")

    # Past path
    if prev_step_range > 0:
        start_step = max(step - prev_step_range, 0)
        past_positions = object_positions[start_step:step]
        past_combined = pv.PolyData()
        for x, y, z in past_positions:
            past_combined = past_combined.merge(pv.Sphere(radius=30, center=(x, y, z)))
        if len(past_positions):
            plotter.add_mesh(past_combined, color="gray", opacity=0.6)
            past_path = np.vstack((past_positions, current_position))
            lines = pv.lines_from_points(past_path)
            plotter.add_mesh(lines, color="gray", line_width=2)

    # Future path
    if future_step_range > 0:
        end_step = min(step + future_step_range, max_steps)
        future_positions = object_positions[step+1:end_step+1]
        future_combined = pv.PolyData()
        for x, y, z in future_positions:
            future_combined = future_combined.merge(pv.Sphere(radius=30, center=(x, y, z)))
        if len(future_positions):
            plotter.add_mesh(future_combined, color="green", opacity=0.6)
            future_path = np.vstack((current_position, future_positions))
            lines = pv.lines_from_points(future_path)
            plotter.add_mesh(lines, color="green", line_width=2)

    # Missed maneuver path
    if show_missed and step < max_steps:
        branch_file = f"Data/ManeuverBranchId{step+5}.csv"
        try:
            branch_df = pd.read_csv(branch_file)
            max_points = len(branch_df)
            clipped_points = min(missed_path_length, max_points)

            pos = np.column_stack((
                branch_df['positionDepRelToChiefLvlhX'][:clipped_points],
                branch_df['positionDepRelToChiefLvlhY'][:clipped_points],
                branch_df['positionDepRelToChiefLvlhZ'][:clipped_points]
            ))


            velocities = branch_df['relativeVelocity'][:clipped_points].values
            lines = pv.lines_from_points(pos)
            lines.cell_data["Velocity"] = velocities[:lines.n_cells]
            
            if match_width_with_threshold:
                tube = lines.tube(radius=collision_threshold)
                plotter.add_mesh(
                    tube,
                    scalars="Velocity",
                    cmap="plasma",
                    opacity=0.5,
                    show_scalar_bar=True,
                )
            else:
                plotter.add_mesh(
                    lines,
                    scalars="Velocity",
                    cmap="plasma",
                    line_width=3,
                    show_scalar_bar=True,
                )


            # Collision detection
            distances = np.linalg.norm(pos - target_position, axis=1)
            min_dist = np.min(distances)
            if min_dist < collision_threshold:
                collision_idx = np.argmin(distances)
                collision_point = pos[collision_idx]
                x_marker = pv.Sphere(radius=80, center=collision_point)
                plotter.add_mesh(x_marker, color="red")
                print(f"⚠️ Collision detected at distance {min_dist:.2f} m")
            else:
                print("✅ No collision detected.")
        except Exception as e:
            print(f"Could not load missed maneuver path for step {step+5}: {e}")
    elif show_missed:
        print("Rendezvous complete, no missed maneuvers to show.")

    plotter.add_axes()
    plotter.reset_camera()
    plotter.show()


# to show full text by sliders
slider_style = {'description_width': 'initial'}

maneuver_select = widgets.IntSlider(
    value=0,
    min=0,
    max=max_steps,
    step=1,
    description='Maneuver Number',
    continuous_update=False,
    style=slider_style
)

previous_maneuver_select = widgets.IntSlider(
    value=0,
    min=0,
    max=max_steps,
    step=1,
    description='Number of Past Maneuvers',
    continuous_update=False,
    style=slider_style
)

future_maneuver_select = widgets.IntSlider(
    value=0,
    min=0,
    max=max_steps,
    step=1,
    description='Number of Future Maneuvers',
    continuous_update=False,
    style=slider_style
)

missed_maneuver_checkbox = widgets.Checkbox(
    value=False,
    description='Show Missed Maneuver Path',
    indent=False
)

missed_path_length_slider = widgets.IntSlider(
    value=500,
    min=10,
    max=5000,
    step=10,
    description='Missed Path Length (points)',
    continuous_update=False,
    style=slider_style
)

collision_threshold_slider = widgets.IntSlider(
    value=100,
    min=10,
    max=1000,
    step=10,
    description='Collision Threshold (m)',
    continuous_update=False,
    style=slider_style
)

match_threshold_checkbox = widgets.Checkbox(
    value=False,
    description='Match width with threshold',
    indent=False
)


widgets.interact(
    plot_maneuver,
    step=maneuver_select,
    prev_step_range=previous_maneuver_select,
    future_step_range=future_maneuver_select,
    show_missed=missed_maneuver_checkbox,
    missed_path_length=missed_path_length_slider,
    collision_threshold=collision_threshold_slider,
    match_width_with_threshold=match_threshold_checkbox
)



interactive(children=(IntSlider(value=0, continuous_update=False, description='Maneuver Number', max=43, style…

<function __main__.plot_maneuver(step, prev_step_range=0, future_step_range=0, show_missed=False, missed_path_length=500, collision_threshold=100, match_width_with_threshold=False)>

In [23]:
def draw_maneuver_scene(plotter, step):
    plotter.add_mesh(stars_combined, color="white", opacity=0.5)
    plotter.add_mesh(target, color="orange", opacity=0.8)


    offset_distance = 13000 - (step * 200)
    elevation = 2000
    
    camera_position = np.array([offset_distance, offset_distance, elevation])
    focal_point = np.array([0.0, 0.0, 0.0])
    view_up = np.array([0, 0, 1])
    
    plotter.camera_position = [camera_position, focal_point, view_up]


    current_position = object_positions[step]
    satellite_copy = satellite_mesh.copy()
    satellite_copy.scale(30.0, inplace=True)
    satellite_copy.translate(current_position, inplace=True)
    plotter.add_mesh(satellite_copy, color="cyan")
    
    if step > 0:
        past_segment = np.array([object_positions[step - 1], current_position])
        plotter.add_lines(past_segment, color="gray", width=2)
    if step < len(object_positions) - 1:
        next_segment = np.array([current_position, object_positions[step + 1]])
        plotter.add_lines(next_segment, color="lime", width=2)


plotter = pv.Plotter(off_screen=True)
plotter.set_background("black")
plotter.open_movie("maneuver_animation.mp4", framerate=3)

for step in range(max_steps + 1):
    plotter.clear()
    draw_maneuver_scene(plotter, step)
    plotter.write_frame()

plotter.close()
print("created animation")


🎉 Done! Animation saved as 'maneuver_animation.mp4'
